In [51]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import difflib
import numpy as np
from joblib import Parallel, delayed

import re

removelist = ".?!"

pattern = r'[^\w'+removelist+']'

remove_list_subject = ""
pattern_subject = r'[^\w'+remove_list_subject+']'

def findRootSubject(subject_title):
    subject_root = subject_title.split(':')[1].strip()
    subject_root = re.sub(pattern_subject, ' ', subject_root)
    print (subject_root)
    root_subjects = df_other_subjects[df_other_subjects['Subject'].str.contains(subject_root)]['Subject'].unique()
    print ('a', root_subjects)
    matched_subjects = \
                [subject for subject in root_subjects if difflib.SequenceMatcher(None, a=subject, b=subject_root).ratio()>0.95]

    return [matched_subjects, subject_title]

In [ ]:
#Preliminary processing

In [111]:
df = pd.read_csv('all_results.csv')

In [112]:
df.shape

(517401, 7)

In [113]:
df.rename(columns={'0':'MessageID', '1':'DateTime', '2':'From', '3':'To', '4':'Subject', '5':'Body'}, inplace=True)

In [114]:
df['DateTime'] = df['DateTime'].apply(lambda x: x[5:-1])

In [115]:
df['From'] = df['From'].apply(lambda x: x.split(':')[1:][0][:-1])
df['To'] = df['To'].apply(lambda x: x.split(':')[1:][0][:-1])

In [ ]:
df['MessageID'] = df['MessageID'].apply(lambda x: x.split(':')[1:][0][:-1])

In [124]:
df['Subject'] = df['Subject'].apply(lambda x: x[9:-1])

In [125]:
df['AccountName'] = df['path'].apply(lambda x: x.split('\\')[4])
df['Folder'] = df['path'].apply(lambda x: x.split('\\')[5])

In [126]:
df.to_csv('all_results_process_stage_1.csv', index=False, encoding='utf-8')

In [ ]:
#Stage 1 processing

In [149]:
df = pd.read_csv('all_results_process_stage_1.csv', encoding='utf-8')

In [150]:
df = df[df['Subject'].isnull() == False]

In [5]:
junk_folders = ['discussion_threads', 'sent_mail', 'all_documents', 'deleted_items']

In [6]:
df_restricted = df[(df['Folder'].isin(junk_folders) == False)]

In [ ]:
###########Stage 1A finished

In [8]:
df_restricted.to_csv('all_results_process_stage_1a.csv', index=False, encoding='utf-8')

In [2]:
df_restricted = pd.read_csv('all_results_process_stage_1a.csv', encoding='utf-8')

In [84]:
df_restricted['DateTime'] = pd.to_datetime(df_restricted['DateTime'])

In [4]:
#Dedupe
df_restricted = df_restricted[df_restricted.duplicated(subset=['DateTime', 'From', 'To', 'Subject']) == False]

df_restricted = df_restricted[df_restricted['To'].str.contains('all.houston')==False]

In [5]:
reply_subjects = df_restricted[(df_restricted['Subject'].str.contains('RE:')) | \
              (df_restricted['Subject'].str.contains('Re:')) | \
              (df_restricted['Subject'].str.contains('re:'))]['Subject'].values

fwd_subjects = df_restricted[(df_restricted['Subject'].str.contains('FW:')) | \
              (df_restricted['Subject'].str.contains('Fw:')) | \
              (df_restricted['Subject'].str.contains('fw:'))]['Subject'].values

In [6]:
thread_names = list(reply_subjects) + list(fwd_subjects)

In [7]:
df_other_subjects = df_restricted[df_restricted['Subject'].isin(thread_names) == False]

other_subjects = df_other_subjects['Subject'].unique()

In [65]:
#list_of_root_subjects = []
#for idx_subject, subject in enumerate(thread_names):
#    if idx_subject<520: continue
#    if idx_subject % 1 == 0:
#        print (idx_subject, subject)
#    subject_root = findRootSubject(subject)
#    if len(subject_root)>0:
#        list_of_root_subjects.append([subject_root, subject])

In [66]:
df_matched = pd.read_csv('root_subject_matching.csv')

In [70]:
df_matched = df_matched[(df_matched['Root'].isnull()==False) & (df_matched['Subject'].isnull()==False)]

In [77]:
df_matched.values[0][1]

'RE: West Position'

In [86]:
df_restricted[df_restricted['Subject'].str.contains('West Position')].sort_values(by=['DateTime'])

,MessageID,DateTime,From,To,Subject,Body,path,AccountName,Folder
135552,Message-ID: <18799265.1075840891880.JavaMail.e...,2001-03-16 22:00:00,tim.belden@enron.com,"louise.kitchen@enron.com, john.lavorato@enron...",West Positions,"['Cc: kimberly.hillis@enron.com\n', 'Mime-Vers...",d:\datasets\maildir\\kitchen-l\_americas\portl...,kitchen-l,_americas
73445,Message-ID: <30173301.1075853109890.JavaMail.e...,2001-10-23 16:12:00,mike.grigsby@enron.com,john.lavorato@enron.com,West Positions,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\38,grigsby-m,sent_items
73238,Message-ID: <6907575.1075853109251.JavaMail.ev...,2001-10-28 22:10:44,mike.grigsby@enron.com,ryan.o'rourke@enron.com,RE: West Positions 1025,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\10,grigsby-m,sent_items
73483,Message-ID: <26191463.1075853109158.JavaMail.e...,2001-10-29 16:24:56,mike.grigsby@enron.com,ryan.o'rourke@enron.com,RE: West Positions 1025,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\6,grigsby-m,sent_items
72739,Message-ID: <9708727.1075853108628.JavaMail.ev...,2001-10-30 00:15:41,ryan.o'rourke@enron.com,mike.grigsby@enron.com,West Positions 1029,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\inbox\2,grigsby-m,inbox
73237,Message-ID: <29375786.1075853109046.JavaMail.e...,2001-10-30 04:15:42,mike.grigsby@enron.com,ryan.o'rourke@enron.com,RE: West Positions 1029,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\1,grigsby-m,sent_items
73412,Message-ID: <27926380.1075862024342.JavaMail.e...,2001-11-02 17:16:53,mike.grigsby@enron.com,ryan.o'rourke@enron.com,RE: West Positions 1101,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\349,grigsby-m,sent_items
73432,Message-ID: <26907042.1075862024812.JavaMail.e...,2001-11-09 19:22:16,mike.grigsby@enron.com,ryan.o'rourke@enron.com,RE: West Positions 1108,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\368,grigsby-m,sent_items
73342,Message-ID: <6609133.1075855352709.JavaMail.ev...,2001-11-26 18:28:29,mike.grigsby@enron.com,ryan.o'rourke@enron.com,RE: West Positions 1121,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\195,grigsby-m,sent_items
546,Message-ID: <24465225.1075855377002.JavaMail.e...,2001-12-05 14:02:16,k..allen@enron.com,matt.smith@enron.com,FW: West Position,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\allen-p\sent_items\16,allen-p,sent_items


In [76]:
df_restricted[df_restricted['Subject'] == 'West Positions']

,MessageID,DateTime,From,To,Subject,Body,path,AccountName,Folder
73445,Message-ID: <30173301.1075853109890.JavaMail.e...,"Tue, 23 Oct 2001 09:12:00 -0700 (PDT)",mike.grigsby@enron.com,john.lavorato@enron.com,West Positions,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\grigsby-m\sent_items\38,grigsby-m,sent_items
135552,Message-ID: <18799265.1075840891880.JavaMail.e...,"Fri, 16 Mar 2001 14:00:00 -0800 (PST)",tim.belden@enron.com,"louise.kitchen@enron.com, john.lavorato@enron...",West Positions,"['Cc: kimberly.hillis@enron.com\n', 'Mime-Vers...",d:\datasets\maildir\\kitchen-l\_americas\portl...,kitchen-l,_americas


In [16]:
findRootSubject(reply_subjects[1])

array(['ion: 1.0',
       "ANCHORDESK: Hope ahead: What I learned from 2001's tragedies",
       'NGI Publications - Monday, December 31st 2001', ...,
       'Trade with John Lavorato', 'Gas Hedges',
       'Calgary Analyst/Associate'], dtype=object)

In [34]:
#df.head()

In [87]:
df_restricted = df_restricted[(df_restricted['From'].str.contains('enron')) & (df_restricted['To'].str.contains('enron'))
                             & (df_restricted['From'].str.contains('mailman')==False)]

In [88]:
subject_counts = df_restricted.groupby('Subject')['MessageID'].count().reset_index()
subject_counts = subject_counts[subject_counts['Subject'].str.contains('@')==False]
thread_subjects = subject_counts[subject_counts['MessageID']!=1]['Subject'].values

In [89]:
df_threads = df_restricted[df_restricted['Subject'].isin(thread_subjects)].sort_values(by=['Subject', 'DateTime'])

In [125]:
def cleanBody(body):
    body_text = body[body.index('X-FileName:'):].split('\\n')[2:]
    #Replace these lines with regex!
    body_text = [sent for sent in body_text if sent[len(sent)-1]=='.' or sent[len(sent)-1]=='!' or sent[len(sent)-1]=='?']
    
    body_text = [sent.replace('\\t', '') for sent in body_text]

    body_text = [re.sub(pattern, ' ', sent) for sent in body_text]
    
    string_body_text = ' '.join(body_text)
    
    return ' '.join(string_body_text.split())

In [126]:
df_threads['CleanBody'] = df_threads['Body'].apply(cleanBody)

In [127]:
df_threads = df_threads[df_threads.duplicated(subset=['CleanBody'])==False]

In [128]:
def findRootSubject(subject_title):
    subject_root = subject_title.split(':')[1].strip()
    subject_root = re.sub(pattern_subject, ' ', subject_root)
    print (subject_root)
    root_subjects = df_other_subjects[df_other_subjects['Subject'].str.contains(subject_root)]['Subject'].unique()
    print ('a', root_subjects)
    matched_subjects = \
                [subject for subject in root_subjects if difflib.SequenceMatcher(None, a=subject, b=subject_root).ratio()>0.95]

    return [matched_subjects, subject_title]

In [129]:
df_threads_sent = df_threads[df_threads['Folder'] == 'sent']

In [130]:
df_subject_counts = df_threads_sent.groupby(['Subject']).count().reset_index()

In [131]:
cleaned_thread_subjects = df_subject_counts[df_subject_counts['MessageID'] >1]['Subject'].unique()

In [132]:
len(cleaned_thread_subjects)

2912

In [134]:
df_threads_sent[df_threads_sent['Subject'] == cleaned_thread_subjects[800]].sort_values(by=['DateTime'])['CleanBody'].values

array(['The credit language is not attached? Enron North America Corp.',
       'not there? Enron North America Corp.'], dtype=object)

In [137]:
df_threads_sent

,MessageID,DateTime,From,To,Subject,Body,path,AccountName,Folder,CleanBody
219534,Message-ID: <15124013.1075844902149.JavaMail.e...,2001-01-03 13:08:00,sara.shackleton@enron.com,jeff.blumenthal@enron.com,"""ECT Europe Accounting"" as ENA hedge counterparty","['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\shackleton-s\sent\5239,shackleton-s,sent,
219583,Message-ID: <9546771.1075844903306.JavaMail.ev...,2001-01-10 09:50:00,sara.shackleton@enron.com,jeff.blumenthal@enron.com,"""ECT Europe Accounting"" as ENA hedge counterparty","['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\shackleton-s\sent\5286,shackleton-s,sent,Enron North America Corp.
219624,Message-ID: <28013602.1075844904192.JavaMail.e...,2001-01-17 09:00:00,sara.shackleton@enron.com,"jeff.blumenthal@enron.com, peter.russell@enro...","""ECT Europe Accounting"" as ENA hedge counterparty","['Cc: laurel.adams@enron.com\n', 'Mime-Version...",d:\datasets\maildir\\shackleton-s\sent\5322,shackleton-s,sent,Enron North America Corp. Enron North America ...
239878,Message-ID: <16322304.1075841886838.JavaMail.e...,2001-01-09 16:48:00,kate.symes@enron.com,christian.yoder@enron.com,#485446,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\symes-k\sent\346,symes-k,sent,a chance? delivery stated herein plus 3.00. Is...
240342,Message-ID: <5551071.1075841896510.JavaMail.ev...,2001-03-05 12:02:00,kate.symes@enron.com,tom.alonso@enron.com,#516506,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\symes-k\sent\764,symes-k,sent,This deal also has firm energy while the count...
240340,Message-ID: <24655390.1075841896465.JavaMail.e...,2001-03-05 11:59:00,kate.symes@enron.com,tom.alonso@enron.com,#517518 and 517520,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\symes-k\sent\762,symes-k,sent,disagree. Please let me know when you get a ch...
240361,Message-ID: <17715502.1075841896900.JavaMail.e...,2001-03-06 11:36:00,kate.symes@enron.com,diana.scholtes@enron.com,#519874.01,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\symes-k\sent\781,symes-k,sent,dec 01 on peak 50 mw DJ Mid C index trade date...
240272,Message-ID: <27586818.1075841895016.JavaMail.e...,2001-02-27 11:40:00,kate.symes@enron.com,lester.rawson@enron.com,#531910,"['Mime-Version: 1.0\n', 'Content-Type: text/pl...",d:\datasets\maildir\\symes-k\sent\700,symes-k,sent,in it. Let me know. We need a contact for this...
247519,Message-ID: <8369055.1075860080361.JavaMail.ev...,2000-09-11 10:54:00,mark.taylor@enron.com,michael.moscoso@enron.com,(00-316) Initial Margins for the Mid-Columbia ...,"[' Contract\n', 'Mime-Version: 1.0\n', 'Conten...",d:\datasets\maildir\\taylor-m\sent\1355,taylor-m,sent,I m not sure if you handle NYMEX margin issues...
96451,Message-ID: <15389568.1075847195642.JavaMail.e...,2000-09-12 08:58:00,tana.jones@enron.com,mary.ruffer@enron.com,(00-316) Initial Margins for the Mid-Columbia ...,"[' Contract\n', 'Mime-Version: 1.0\n', 'Conten...",d:\datasets\maildir\\jones-t\sent\1873,jones-t,sent,trading day. trading day. McCoy at 212 299 292...


In [147]:
df_threads_sent.sort_values(by=['Subject', 'DateTime'], inplace=True)

In [150]:
df_threads_sent = df_threads_sent[(df_threads_sent['CleanBody'].isnull()==False) & 
                                  (df_threads_sent['CleanBody'].apply(lambda x: len(x))>0) ]

In [152]:
df_threads_sent.sort_values(by=['Subject', 'DateTime']).to_csv('Enron_Email_Threads_Clean_v1.csv', index=False, encoding='utf-8')

In [153]:
df_threads_sent.shape

(12044, 10)